# Sequence-to-sequence (seq2seq) models and machine translation

The `17_seq2seq_translation` notebook focuses on sequence-to-sequence (seq2seq) models for machine translation, a key application of neural networks in natural language processing. It covers preparing a dataset for translation tasks, building both the Encoder and Decoder models, and combining them into a complete seq2seq architecture. 

The notebook further explores training the model, evaluating its performance, translating new sentences, and experimenting with hyperparameters to fine-tune the model’s accuracy and fluency in translation tasks.

## Table of contents

1. [Understanding seq2seq models and machine translation](#understanding-seq2seq-models-and-machine-translation)
2. [Setting up the environment](#setting-up-the-environment)
3. [Preparing the dataset for machine translation](#preparing-the-dataset-for-machine-translation)
4. [Building the Encoder model](#building-the-encoder-model)
5. [Building the Decoder model](#building-the-decoder-model)
6. [Combining Encoder and Decoder into a seq2seq model](#combining-encoder-and-decoder-into-a-seq2seq-model)
7. [Training the seq2seq model](#training-the-seq2seq-model)
8. [Evaluating the seq2seq model](#evaluating-the-seq2seq-model)
9. [Translating new sentences](#translating-new-sentences)
10. [Experimenting with hyperparameters](#experimenting-with-hyperparameters)

## Understanding seq2seq models and machine translation

Sequence-to-sequence (seq2seq) models are a class of neural networks designed to transform one sequence into another, making them particularly effective for tasks where input and output are sequences of varying lengths. These models are widely used in tasks like **machine translation**, where the input is a sentence in one language and the output is the translation in another language. Other applications include text summarization, speech recognition, and image captioning.

The seq2seq model is based on a **recurrent neural network (RNN)** architecture and is typically composed of two main parts: an **encoder** and a **decoder**. The encoder processes the input sequence and compresses it into a fixed-size context vector, and the decoder takes this context vector to generate the output sequence.

### **Key concepts**
Sequence-to-sequence (seq2seq) models are a class of neural networks designed to map input sequences to output sequences of variable lengths, making them ideal for tasks like machine translation, text summarization, and conversational AI. These models typically consist of two main components: an **encoder** and a **decoder**.

- **Encoder**: Processes the input sequence and encodes it into a fixed-length context vector that captures the semantic information.
- **Context Vector**: Acts as a bridge between the encoder and decoder, summarizing the input sequence.
- **Decoder**: Generates the output sequence one token at a time, conditioned on the context vector and its own previous outputs.

Training seq2seq models involves teacher forcing, where the true output tokens are fed into the decoder during training to improve convergence. These models often use Recurrent Neural Networks (RNNs), Long Short-Term Memory (LSTM) networks, or Gated Recurrent Units (GRUs) as their core architecture.

### **Applications**
Seq2seq models are widely used in sequence transformation tasks, such as:
- **Machine translation**: Translating text from one language to another (e.g., English to French).
- **Text summarization**: Condensing long documents into shorter summaries.
- **Conversational AI**: Powering chatbots and virtual assistants for natural dialogue generation.
- **Speech recognition**: Converting audio signals into text sequences.
- **Code generation**: Translating natural language descriptions into code snippets.

### **Advantages**
- **Variable-length mapping**: Handles sequences of differing lengths for input and output.
- **General-purpose framework**: Adaptable to a wide range of sequence transformation tasks.
- **Sequential dependency capture**: Models relationships across input and output tokens effectively.
- **Scalability**: Can be extended with attention mechanisms or transformer architectures for improved performance.

### **Challenges**
- **Information bottleneck**: Compressing an entire input sequence into a single context vector can lead to loss of information, especially for long sequences.
- **Long-term dependency modeling**: Traditional RNN-based seq2seq models struggle with capturing dependencies in lengthy sequences.
- **Data requirements**: Requires large and high-quality parallel datasets for effective training.
- **Training instability**: Susceptible to issues like vanishing gradients, especially with deeper architectures.

## Setting up the environment


##### **Q1: How do you install the necessary libraries for building and training seq2seq models in PyTorch?**


In [1]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# !pip install numpy matplotlib scikit-learn pandas
# !pip install nltk

##### **Q2: How do you import the required modules for model building, training, and data loading in PyTorch?**


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import random
import numpy as np
import nltk
import os
import urllib.request
import gzip
import shutil
import re

##### **Q3: How do you set up the environment to use a GPU for training seq2seq models, and how do you fallback to CPU in PyTorch?**


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


##### **Q4: How do you set random seeds in PyTorch to ensure reproducibility when training seq2seq models?**

In [4]:
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)

## Preparing the dataset for machine translation


##### **Q5: How do you load a machine translation dataset to use in PyTorch?**


In [6]:
nltk.download('punkt')

url_train_en = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/train.en.gz'
url_train_de = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/train.de.gz'

os.makedirs('data', exist_ok=True)

def download_and_extract(url, filename):
    filepath = os.path.join('data', filename)
    if not os.path.exists(filepath):
        urllib.request.urlretrieve(url, filepath + '.gz')
        with gzip.open(filepath + '.gz', 'rb') as f_in:
            with open(filepath, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        os.remove(filepath + '.gz')

download_and_extract(url_train_en, 'train.en')
download_and_extract(url_train_de, 'train.de')

[nltk_data] Downloading package punkt to /home/fellmir/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


##### **Q6: How do you preprocess the dataset by tokenizing the sentences and converting them into sequences of indices?**


In [10]:
def simple_tokenizer(text):
    return re.findall(r'\b\w+\b', text.lower())

In [11]:
# Assign the tokenizer functions:
tokenizer_src = simple_tokenizer  # For English
tokenizer_trg = simple_tokenizer  # For German

with open('data/train.en', 'r', encoding='utf-8') as f:
    sentences_en = f.readlines()

with open('data/train.de', 'r', encoding='utf-8') as f:
    sentences_de = f.readlines()

assert len(sentences_en) == len(sentences_de)  # Ensure both files have the same number of sentences

tokenized_en = [tokenizer_src(sentence) for sentence in sentences_en]
tokenized_de = [tokenizer_trg(sentence) for sentence in sentences_de]  # Tokenize the sentences

In [ ]:
# Alternative using NLTK's wordpunct_tokenize:
# from nltk.tokenize import wordpunct_tokenize

# tokenizer_src = wordpunct_tokenize  # For English
# tokenizer_trg = wordpunct_tokenize  # For German

# Tokenize the sentences
# tokenized_en = [tokenizer_src(sentence.lower()) for sentence in sentences_en]
# tokenized_de = [tokenizer_trg(sentence.lower()) for sentence in sentences_de]

##### **Q7: How do you build vocabulary for both the source and target languages?**


In [12]:
from collections import Counter

def build_vocab(tokenized_sentences, min_freq):
    counter = Counter()
    for tokens in tokenized_sentences:
        counter.update(tokens)
    vocab = {'<unk>': 0, '<pad>': 1, '<bos>': 2, '<eos>': 3}
    idx = 4
    for word, freq in counter.items():
        if freq >= min_freq:
            vocab[word] = idx
            idx += 1
    return vocab

MIN_FREQ = 2
vocab_src = build_vocab(tokenized_en, MIN_FREQ)
vocab_trg = build_vocab(tokenized_de, MIN_FREQ)

inv_vocab_src = {idx: word for word, idx in vocab_src.items()}
inv_vocab_trg = {idx: word for word, idx in vocab_trg.items()}  # Inverse vocabularies for decoding

##### **Q8: How do you create DataLoaders for batching the source-target sentence pairs during training?**

In [13]:
class TranslationDataset(Dataset):
    def __init__(self, tokenized_src, tokenized_trg, vocab_src, vocab_trg):
        self.data = []
        for src_tokens, trg_tokens in zip(tokenized_src, tokenized_trg):
            src_indices = [vocab_src.get('<bos>')] + [vocab_src.get(token, vocab_src['<unk>']) for token in src_tokens] + [vocab_src.get('<eos>')]
            trg_indices = [vocab_trg.get('<bos>')] + [vocab_trg.get(token, vocab_trg['<unk>']) for token in trg_tokens] + [vocab_trg.get('<eos>')]
            self.data.append((torch.tensor(src_indices), torch.tensor(trg_indices)))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

def collate_fn(batch):
    src_batch, trg_batch = zip(*batch)
    src_padded = nn.utils.rnn.pad_sequence(src_batch, padding_value=vocab_src['<pad>'], batch_first=True)
    trg_padded = nn.utils.rnn.pad_sequence(trg_batch, padding_value=vocab_trg['<pad>'], batch_first=True)
    return src_padded, trg_padded

In [14]:
from sklearn.model_selection import train_test_split

train_src, valid_src, train_trg, valid_trg = train_test_split(tokenized_en, tokenized_de, test_size=0.1, random_state=42)

train_dataset = TranslationDataset(train_src, train_trg, vocab_src, vocab_trg)
valid_dataset = TranslationDataset(valid_src, valid_trg, vocab_src, vocab_trg)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

## Building the Encoder model


##### **Q9: How do you define the architecture of the Encoder model using PyTorch’s `nn.Module`?**


In [15]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, batch_first=True)
        
    def forward(self, src):
        embedded = self.embedding(src)
        outputs, hidden = self.rnn(embedded)
        return hidden

##### **Q10: How do you implement the forward pass of the Encoder to process input sequences and generate the context vector?**


In [16]:
# The forward pass of the Encoder processes input sequences and generates the context vector (hidden state)

##### **Q11: How do you specify the number of layers and hidden units in the Encoder?**

In [17]:
INPUT_DIM = len(vocab_src)
ENC_EMB_DIM = 256
HID_DIM = 512
ENC_N_LAYERS = 2

encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_N_LAYERS).to(device)  # Initialize the Encoder

## Building the Decoder model


##### **Q12: How do you define the Decoder architecture using PyTorch’s `nn.Module`?**


In [18]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, batch_first=True)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
    def forward(self, input, hidden):
        input = input.unsqueeze(1)
        embedded = self.embedding(input)
        output, hidden = self.rnn(embedded, hidden)
        prediction = self.fc_out(output.squeeze(1))
        return prediction, hidden

##### **Q13: How do you implement the forward pass of the Decoder to generate translated sequences from the context vector?**


In [ ]:
# The forward pass of the Decoder generates translated sequences from the context vector

##### **Q14: How do you use the `nn.Linear` and `nn.Softmax` layers to convert the Decoder's output into predicted tokens?**

In [ ]:
# The nn.Linear and nn.Softmax layers convert the Decoder's output into predicted tokens

## Combining Encoder and Decoder into a seq2seq model


##### **Q15: How do you combine the Encoder and Decoder models into a complete seq2seq model for machine translation?**


In [19]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.embedding.num_embeddings
        
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)
        
        hidden = self.encoder(src)
        
        input = trg[:, 0]
        
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden)
            outputs[:, t, :] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[:, t] if teacher_force else top1
        return outputs

In [20]:
OUTPUT_DIM = len(vocab_trg)
DEC_EMB_DIM = 256
DEC_N_LAYERS = 2

decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_N_LAYERS).to(device)
seq2seq_model = Seq2Seq(encoder, decoder, device).to(device)

##### **Q16: How do you implement teacher forcing in the training loop to improve the Decoder’s performance during training?**


In [21]:
# Teacher forcing is implemented in the Seq2Seq model's forward method

##### **Q17: How do you implement the forward pass for the combined seq2seq model, using the context vector from the Encoder to initialize the Decoder?**

In [22]:
# The forward pass for the combined seq2seq model uses the context vector from the Encoder to initialize the Decoder

## Training the seq2seq model


##### **Q18: How do you define the loss function for training the seq2seq model on sequence data?**


In [23]:
PAD_IDX = vocab_trg['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)  # Define the loss function with padding index ignored

##### **Q19: How do you configure an optimizer to update the parameters of both the Encoder and Decoder models during training?**


In [24]:
optimizer = optim.Adam(seq2seq_model.parameters())

##### **Q20: How do you implement the training loop for the seq2seq model, including the forward pass, loss calculation, and backpropagation?**


In [26]:
num_epochs = 30

for epoch in range(num_epochs):
    seq2seq_model.train()
    epoch_loss = 0
    for src_batch, trg_batch in train_loader:
        src_batch = src_batch.to(device)
        trg_batch = trg_batch.to(device)
        
        optimizer.zero_grad()
        output = seq2seq_model(src_batch, trg_batch)
        
        # output: (batch_size, trg_len, trg_vocab_size)
        output_dim = output.shape[-1]
        output = output[:, 1:, :].reshape(-1, output_dim)
        trg = trg_batch[:, 1:].reshape(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}')

Epoch 10/30, Loss: 0.7457
Epoch 20/30, Loss: 0.6495
Epoch 30/30, Loss: 0.6860


##### **Q21: How do you monitor and log the training loss over epochs to ensure the seq2seq model is learning effectively?**

In [ ]:
# Training loss is monitored and logged in the training loop above

## Evaluating the seq2seq model


##### **Q22: How do you evaluate the seq2seq model on a validation dataset using metrics such as the BLEU score?**


In [27]:
from nltk.translate.bleu_score import corpus_bleu

def evaluate(model, data_loader):
    model.eval()
    references = []
    hypotheses = []
    
    with torch.no_grad():
        for src_batch, trg_batch in data_loader:
            src_batch = src_batch.to(device)
            trg_batch = trg_batch.to(device)
            output = model(src_batch, trg_batch, teacher_forcing_ratio=0)
            output_tokens = output.argmax(2)
            for i in range(src_batch.size(0)):
                trg_indices = trg_batch[i].cpu().numpy()
                output_indices = output_tokens[i].cpu().numpy()
                trg_tokens = [inv_vocab_trg[idx] for idx in trg_indices if idx != PAD_IDX and idx != vocab_trg['<bos>']]
                output_tokens_list = [inv_vocab_trg[idx] for idx in output_indices if idx != PAD_IDX and idx != vocab_trg['<bos>']]
                references.append([trg_tokens])
                hypotheses.append(output_tokens_list)
    bleu = corpus_bleu(references, hypotheses)
    return bleu

In [28]:
bleu_score = evaluate(seq2seq_model, valid_loader)
print(f'Validation BLEU score: {bleu_score*100:.2f}')

Validation BLEU score: 7.99


##### **Q23: How do you implement a function to calculate the BLEU score to assess the quality of the machine-translated sequences?**


In [ ]:
# The "evaluate" function calculates the BLEU score to assess translation quality

##### **Q24: How do you compare the model's predictions to the target translations during evaluation to measure performance?**

In [ ]:
# The model's predictions are compared to the target translations during evaluation

## Translating new sentences


##### **Q25: How do you implement a function to translate new sentences using the trained seq2seq model?**


In [29]:
def translate_sentence(sentence, vocab_src, vocab_trg, model, tokenizer_src, max_len=50):
    model.eval()
    tokens = ['<bos>'] + tokenizer_src(sentence.lower()) + ['<eos>']
    src_indices = [vocab_src.get(token, vocab_src['<unk>']) for token in tokens]
    src_tensor = torch.LongTensor(src_indices).unsqueeze(0).to(device)
    
    with torch.no_grad():
        hidden = model.encoder(src_tensor)
    
    input_token = torch.LongTensor([vocab_trg['<bos>']]).to(device)
    outputs = []
    
    for _ in range(max_len):
        output, hidden = model.decoder(input_token, hidden)
        top1 = output.argmax(1)
        outputs.append(top1.item())
        if top1.item() == vocab_trg['<eos>']:
            break
        input_token = top1
    translated_tokens = [inv_vocab_trg.get(idx, '<unk>') for idx in outputs]
    return translated_tokens

##### **Q26: How do you handle sentences of varying lengths when translating new sentences with the seq2seq model?**


In [ ]:
# The translate_sentence function handles sentences of varying lengths using a max_len parameter

##### **Q27: How do you visualize the original, translated, and reference (ground truth) sentences to evaluate the model’s translation performance?**

In [30]:
def display_translation(sentence):
    print(f'Original: {sentence}')
    translation = translate_sentence(sentence, vocab_src, vocab_trg, seq2seq_model, tokenizer_src)
    print(f'Translated: {" ".join(translation)}')

In [31]:
test_sentence = "A man is playing a guitar."
display_translation(test_sentence)

Original: A man is playing a guitar.
Translated: ein mann spielt gitarre einer trommel <eos>


In [32]:
display_translation('the book is on the table')

Original: the book is on the table
Translated: die am tisch beim essen <eos>


In [37]:
display_translation('words')

Original: words
Translated: straßenkünstler <unk> die arbeit <eos>


## Experimenting with hyperparameters


##### **Q28: How do you adjust the learning rate and observe its effect on the seq2seq model’s training stability and performance?**


In [38]:
learning_rates = [0.001, 0.0005, 0.0001]

def train_with_learning_rate(lr):
    print(f"\nTraining with learning rate: {lr}")
    # Re-initialize the model:
    encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_N_LAYERS).to(device)
    decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_N_LAYERS).to(device)
    seq2seq_model = Seq2Seq(encoder, decoder, device).to(device)
    
    optimizer = optim.Adam(seq2seq_model.parameters(), lr=lr)
    
    criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
    
    num_epochs = 10
    for epoch in range(num_epochs):
        seq2seq_model.train()
        epoch_loss = 0
        for src_batch, trg_batch in train_loader:
            src_batch = src_batch.to(device)
            trg_batch = trg_batch.to(device)
            
            optimizer.zero_grad()
            output = seq2seq_model(src_batch, trg_batch)
            
            # output: (batch_size, trg_len, trg_vocab_size)
            output_dim = output.shape[-1]
            output = output[:, 1:, :].reshape(-1, output_dim)
            trg = trg_batch[:, 1:].reshape(-1)
            
            loss = criterion(output, trg)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(train_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')
    
    bleu_score = evaluate(seq2seq_model, valid_loader)
    print(f'Validation BLEU score: {bleu_score*100:.2f}')
    return avg_loss, bleu_score

In [39]:
results_lr = []
for lr in learning_rates:
    avg_loss, bleu = train_with_learning_rate(lr)
    results_lr.append({'learning_rate': lr, 'loss': avg_loss, 'bleu_score': bleu})

print("\nLearning Rate Experiment Results:")
for res in results_lr:
    print(f"Learning Rate: {res['learning_rate']}, Final Loss: {res['loss']:.4f}, BLEU Score: {res['bleu_score']*100:.2f}")


Training with learning rate: 0.001
Epoch 1/10, Loss: 4.6847
Epoch 2/10, Loss: 3.6639
Epoch 3/10, Loss: 3.1361
Epoch 4/10, Loss: 2.7249
Epoch 5/10, Loss: 2.3774
Epoch 6/10, Loss: 2.0795
Epoch 7/10, Loss: 1.8155
Epoch 8/10, Loss: 1.6143
Epoch 9/10, Loss: 1.4452
Epoch 10/10, Loss: 1.2949
Validation BLEU score: 9.00

Training with learning rate: 0.0005
Epoch 1/10, Loss: 4.8586
Epoch 2/10, Loss: 3.7968
Epoch 3/10, Loss: 3.2709
Epoch 4/10, Loss: 2.8791
Epoch 5/10, Loss: 2.5498
Epoch 6/10, Loss: 2.2679
Epoch 7/10, Loss: 1.9957
Epoch 8/10, Loss: 1.7537
Epoch 9/10, Loss: 1.5253
Epoch 10/10, Loss: 1.3123
Validation BLEU score: 10.03

Training with learning rate: 0.0001
Epoch 1/10, Loss: 5.4526
Epoch 2/10, Loss: 4.8022
Epoch 3/10, Loss: 4.4589
Epoch 4/10, Loss: 4.1844
Epoch 5/10, Loss: 3.9739
Epoch 6/10, Loss: 3.7985
Epoch 7/10, Loss: 3.6333
Epoch 8/10, Loss: 3.4942
Epoch 9/10, Loss: 3.3708
Epoch 10/10, Loss: 3.2538
Validation BLEU score: 6.85

Learning Rate Experiment Results:
Learning Rate: 0.

##### **Q29: How do you experiment with different batch sizes to observe how they impact training speed and memory usage?**


In [40]:
batch_sizes = [16, 32, 64]

def train_with_batch_size(batch_size):
    print(f"\nTraining with batch size: {batch_size}")
    # Re-create DataLoaders with specified batch size:
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    
    encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_N_LAYERS).to(device)
    decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_N_LAYERS).to(device)
    seq2seq_model = Seq2Seq(encoder, decoder, device).to(device)
    
    optimizer = optim.Adam(seq2seq_model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
    
    num_epochs = 10
    import time
    start_time = time.time()
    for epoch in range(num_epochs):
        seq2seq_model.train()
        epoch_loss = 0
        for src_batch, trg_batch in train_loader:
            src_batch = src_batch.to(device)
            trg_batch = trg_batch.to(device)
            
            optimizer.zero_grad()
            output = seq2seq_model(src_batch, trg_batch)
            
            # output: (batch_size, trg_len, trg_vocab_size)
            output_dim = output.shape[-1]
            output = output[:, 1:, :].reshape(-1, output_dim)
            trg = trg_batch[:, 1:].reshape(-1)
            
            loss = criterion(output, trg)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(train_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')
    end_time = time.time()
    total_time = end_time - start_time
    
    bleu_score = evaluate(seq2seq_model, valid_loader)
    print(f'Validation BLEU score: {bleu_score*100:.2f}')
    print(f"Training time: {total_time:.2f} seconds")
    return avg_loss, bleu_score, total_time

In [41]:
results_bs = []
for batch_size in batch_sizes:
    avg_loss, bleu, total_time = train_with_batch_size(batch_size)
    results_bs.append({'batch_size': batch_size, 'loss': avg_loss, 'bleu_score': bleu, 'training_time': total_time})

print("\nBatch Size Experiment Results:")
for res in results_bs:
    print(f"Batch Size: {res['batch_size']}, Final Loss: {res['loss']:.4f}, BLEU Score: {res['bleu_score']*100:.2f}, Training Time: {res['training_time']:.2f} seconds")


Training with batch size: 16
Epoch 1/10, Loss: 4.4521
Epoch 2/10, Loss: 3.3737
Epoch 3/10, Loss: 2.8145
Epoch 4/10, Loss: 2.3840
Epoch 5/10, Loss: 2.0419
Epoch 6/10, Loss: 1.8051
Epoch 7/10, Loss: 1.6383
Epoch 8/10, Loss: 1.5304
Epoch 9/10, Loss: 1.4372
Epoch 10/10, Loss: 1.3601
Validation BLEU score: 9.88
Training time: 829.96 seconds

Training with batch size: 32
Epoch 1/10, Loss: 4.6968
Epoch 2/10, Loss: 3.6638
Epoch 3/10, Loss: 3.1340
Epoch 4/10, Loss: 2.7402
Epoch 5/10, Loss: 2.3981
Epoch 6/10, Loss: 2.0969
Epoch 7/10, Loss: 1.8206
Epoch 8/10, Loss: 1.6418
Epoch 9/10, Loss: 1.4646
Epoch 10/10, Loss: 1.3230
Validation BLEU score: 8.94
Training time: 537.01 seconds

Training with batch size: 64
Epoch 1/10, Loss: 5.1268
Epoch 2/10, Loss: 4.1523
Epoch 3/10, Loss: 3.6510
Epoch 4/10, Loss: 3.2843
Epoch 5/10, Loss: 2.9597
Epoch 6/10, Loss: 2.7052
Epoch 7/10, Loss: 2.4615
Epoch 8/10, Loss: 2.2732
Epoch 9/10, Loss: 2.0692
Epoch 10/10, Loss: 1.8595
Validation BLEU score: 7.67
Training time

##### **Q30: How do you modify the number of training epochs and analyze how it affects the model’s convergence and translation accuracy?**


In [42]:
epoch_numbers = [5, 15, 20]

def train_with_epochs(num_epochs):
    print(f"\nTraining with number of epochs: {num_epochs}")
    encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_N_LAYERS).to(device)
    decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_N_LAYERS).to(device)
    seq2seq_model = Seq2Seq(encoder, decoder, device).to(device)
    
    optimizer = optim.Adam(seq2seq_model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
    
    for epoch in range(num_epochs):
        seq2seq_model.train()
        epoch_loss = 0
        for src_batch, trg_batch in train_loader:
            src_batch = src_batch.to(device)
            trg_batch = trg_batch.to(device)
            
            optimizer.zero_grad()
            output = seq2seq_model(src_batch, trg_batch)
            
            # output: (batch_size, trg_len, trg_vocab_size)
            output_dim = output.shape[-1]
            output = output[:, 1:, :].reshape(-1, output_dim)
            trg = trg_batch[:, 1:].reshape(-1)
            
            loss = criterion(output, trg)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(train_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')
    
    bleu_score = evaluate(seq2seq_model, valid_loader)
    print(f'Validation BLEU score after {num_epochs} epochs: {bleu_score*100:.2f}')
    return avg_loss, bleu_score

In [43]:
results_epochs = []
for num_epochs in epoch_numbers:
    avg_loss, bleu = train_with_epochs(num_epochs)
    results_epochs.append({'num_epochs': num_epochs, 'loss': avg_loss, 'bleu_score': bleu})

print("\nEpoch Number Experiment Results:")
for res in results_epochs:
    print(f"Number of Epochs: {res['num_epochs']}, Final Loss: {res['loss']:.4f}, BLEU Score: {res['bleu_score']*100:.2f}")


Training with number of epochs: 5
Epoch 1/5, Loss: 4.6984
Epoch 2/5, Loss: 3.6317
Epoch 3/5, Loss: 3.0833
Epoch 4/5, Loss: 2.6554
Epoch 5/5, Loss: 2.2912
Validation BLEU score after 5 epochs: 9.15

Training with number of epochs: 15
Epoch 1/15, Loss: 4.7550
Epoch 2/15, Loss: 3.7759
Epoch 3/15, Loss: 3.2685
Epoch 4/15, Loss: 2.8572
Epoch 5/15, Loss: 2.5082
Epoch 6/15, Loss: 2.1977
Epoch 7/15, Loss: 1.9555
Epoch 8/15, Loss: 1.7371
Epoch 9/15, Loss: 1.5629
Epoch 10/15, Loss: 1.4165
Epoch 11/15, Loss: 1.2908
Epoch 12/15, Loss: 1.1703
Epoch 13/15, Loss: 1.0776
Epoch 14/15, Loss: 1.0027
Epoch 15/15, Loss: 0.9374
Validation BLEU score after 15 epochs: 8.78

Training with number of epochs: 20
Epoch 1/20, Loss: 4.7713
Epoch 2/20, Loss: 3.7686
Epoch 3/20, Loss: 3.2649
Epoch 4/20, Loss: 2.8748
Epoch 5/20, Loss: 2.5597
Epoch 6/20, Loss: 2.2812
Epoch 7/20, Loss: 2.0288
Epoch 8/20, Loss: 1.8197
Epoch 9/20, Loss: 1.6428
Epoch 10/20, Loss: 1.5024
Epoch 11/20, Loss: 1.3940
Epoch 12/20, Loss: 1.2498
Ep

##### **Q31: How do you experiment with different recurrent layers to evaluate their impact on translation quality?**

In [44]:
def train_model(model_type='GRU'):
    print(f"\nTraining with {model_type} model")
    if model_type == 'GRU':
        # Define GRU-based model:
        encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_N_LAYERS).to(device)
        decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_N_LAYERS).to(device)
        seq2seq_model = Seq2Seq(encoder, decoder, device).to(device)
    elif model_type == 'LSTM':
        # Define LSTM-based model:
        class EncoderLSTM(nn.Module):
            def __init__(self, input_dim, emb_dim, hid_dim, n_layers):
                super(EncoderLSTM, self).__init__()
                self.embedding = nn.Embedding(input_dim, emb_dim)
                self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, batch_first=True)
                
            def forward(self, src):
                embedded = self.embedding(src)
                outputs, (hidden, cell) = self.rnn(embedded)
                return hidden, cell
        
        class DecoderLSTM(nn.Module):
            def __init__(self, output_dim, emb_dim, hid_dim, n_layers):
                super(DecoderLSTM, self).__init__()
                self.embedding = nn.Embedding(output_dim, emb_dim)
                self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, batch_first=True)
                self.fc_out = nn.Linear(hid_dim, output_dim)
                
            def forward(self, input, hidden, cell):
                input = input.unsqueeze(1)
                embedded = self.embedding(input)
                output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
                prediction = self.fc_out(output.squeeze(1))
                return prediction, hidden, cell
        
        class Seq2SeqLSTM(nn.Module):
            def __init__(self, encoder, decoder, device):
                super(Seq2SeqLSTM, self).__init__()
                self.encoder = encoder
                self.decoder = decoder
                self.device = device
                
            def forward(self, src, trg, teacher_forcing_ratio=0.5):
                batch_size = src.shape[0]
                trg_len = trg.shape[1]
                trg_vocab_size = self.decoder.embedding.num_embeddings
                
                outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)
                
                hidden, cell = self.encoder(src)
                
                input = trg[:, 0]
                
                for t in range(1, trg_len):
                    output, hidden, cell = self.decoder(input, hidden, cell)
                    outputs[:, t, :] = output
                    teacher_force = random.random() < teacher_forcing_ratio
                    top1 = output.argmax(1)
                    input = trg[:, t] if teacher_force else top1
                return outputs
        
        encoder = EncoderLSTM(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_N_LAYERS).to(device)
        decoder = DecoderLSTM(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_N_LAYERS).to(device)
        seq2seq_model = Seq2SeqLSTM(encoder, decoder, device).to(device)
    else:
        raise ValueError("model_type must be 'GRU' or 'LSTM'")
    
    optimizer = optim.Adam(seq2seq_model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
    
    num_epochs = 10
    for epoch in range(num_epochs):
        seq2seq_model.train()
        epoch_loss = 0
        for src_batch, trg_batch in train_loader:
            src_batch = src_batch.to(device)
            trg_batch = trg_batch.to(device)
            
            optimizer.zero_grad()
            output = seq2seq_model(src_batch, trg_batch)
            
            # output: (batch_size, trg_len, trg_vocab_size)
            output_dim = output.shape[-1]
            output = output[:, 1:, :].reshape(-1, output_dim)
            trg = trg_batch[:, 1:].reshape(-1)
            
            loss = criterion(output, trg)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(train_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')
    
    bleu_score = evaluate(seq2seq_model, valid_loader)
    print(f'Validation BLEU score: {bleu_score*100:.2f}')
    return avg_loss, bleu_score

In [45]:
gru_loss, gru_bleu = train_model(model_type='GRU')  # Train and evaluate GRU-based model

lstm_loss, lstm_bleu = train_model(model_type='LSTM')  # Train and evaluate LSTM-based model

print("\nRecurrent Layer Experiment Results:")
print(f"GRU Model - Final Loss: {gru_loss:.4f}, BLEU Score: {gru_bleu*100:.2f}")
print(f"LSTM Model - Final Loss: {lstm_loss:.4f}, BLEU Score: {lstm_bleu*100:.2f}")


Training with GRU model
Epoch 1/10, Loss: 4.6459
Epoch 2/10, Loss: 3.5893
Epoch 3/10, Loss: 3.0096
Epoch 4/10, Loss: 2.5689
Epoch 5/10, Loss: 2.2054
Epoch 6/10, Loss: 1.8771
Epoch 7/10, Loss: 1.6052
Epoch 8/10, Loss: 1.4044
Epoch 9/10, Loss: 1.2310
Epoch 10/10, Loss: 1.0996
Validation BLEU score: 9.33

Training with LSTM model
Epoch 1/10, Loss: 4.9880
Epoch 2/10, Loss: 4.1204
Epoch 3/10, Loss: 3.7079
Epoch 4/10, Loss: 3.3919
Epoch 5/10, Loss: 3.1239
Epoch 6/10, Loss: 2.8745
Epoch 7/10, Loss: 2.6470
Epoch 8/10, Loss: 2.4340
Epoch 9/10, Loss: 2.2234
Epoch 10/10, Loss: 2.0312
Validation BLEU score: 8.62

Recurrent Layer Experiment Results:
GRU Model - Final Loss: 1.0996, BLEU Score: 9.33
LSTM Model - Final Loss: 2.0312, BLEU Score: 8.62


In [46]:
import shutil
import os

if os.path.exists('data'):
    shutil.rmtree('data')
    print("Folder 'data' has been deleted.")
else:
    print("Folder 'data' does not exist.")

Folder 'data' has been deleted.
